In [0]:
# Set configuration settings
config = {
  "spark.cosmos.accountEndpoint": "https://democosmosnosql.documents.azure.com:443/",
  "spark.cosmos.accountKey": "lRzLNbbsfXgSI4Y7cl6GAqbXITP1GgcNWqRU28MzBVjLJxDBf34rYEmWf9dazOoOHXH56ZZtprHQACDbtMW3Sg==",
  "spark.cosmos.database": "cosmicworks",
  "spark.cosmos.container": "products"
}

# # Configure Catalog Api    
spark.conf.set("spark.sql.catalog.cosmosCatalog", "com.azure.cosmos.spark.CosmosCatalog")
spark.conf.set("spark.sql.catalog.cosmosCatalog.spark.cosmos.accountEndpoint", config["spark.cosmos.accountEndpoint"])
spark.conf.set("spark.sql.catalog.cosmosCatalog.spark.cosmos.accountKey", config["spark.cosmos.accountKey"])

In [0]:

spark.sql(
    "CREATE TABLE IF NOT EXISTS cosmosCatalog.cosmicworks.products "
    "USING cosmos.oltp "
    "TBLPROPERTIES(partitionKeyPath = '/category')"
)
spark.sql(
    "CREATE TABLE IF NOT EXISTS cosmosCatalog.cosmicworks.employees "
    "USING cosmos.oltp "
    "TBLPROPERTIES(partitionKeyPath = '/organization,/department,/team')"
)

DataFrame[]

In [0]:
# Create a database by using the Catalog API    
# Create a database by using the Catalog API    
spark.sql(f"CREATE DATABASE IF NOT EXISTS cosmosCatalog.cosmicworks;")
# Create a products container by using the Catalog API
spark.sql(
    "CREATE TABLE IF NOT EXISTS cosmosCatalog.cosmicworks.products "
    "USING cosmos.oltp "
    "TBLPROPERTIES(partitionKeyPath = '/category')"
)
# Create an employees container by using the Catalog API
spark.sql(
    "CREATE TABLE IF NOT EXISTS cosmosCatalog.cosmicworks.employees "
    "USING cosmos.oltp "
    "TBLPROPERTIES(partitionKeyPath = '/organization,/department,/team')"
)

DataFrame[]

In [0]:
# Create sample data    
products = (
  ("68719518391", "gear-surf-surfboards", "Yamba Surfboard", 12, 850.00, False),
  ("68719518371", "gear-surf-surfboards", "Kiama Classic Surfboard", 25, 790.00, True),
  ("1", "gear-surf-surfboards", "Yamba Surfboard", 12, 850.00, False),
  ("2", "gear-surf-surfboards", "Kiama Classic Surfboard", 25, 790.00, True)
)

In [0]:
# Ingest sample data    
spark.createDataFrame(products) \
  .toDF("id", "category", "name", "quantity", "price", "clearance") \
  .write \
  .format("cosmos.oltp") \
  .options(**config) \
  .option("spark.cosmos.database", "cosmicworks") \
  .option("spark.cosmos.container", "products")\
  .mode("APPEND") \
  .save()

In [0]:
# Load data    
df = spark.read.format("cosmos.oltp") \
  .options(**config) \
  .option("spark.cosmos.read.inferSchema.enabled", "true") \
  .load()

In [0]:
# Render schema    
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- clearance: boolean (nullable = true)
 |-- price: integer (nullable = true)
 |-- id: string (nullable = false)
 |-- category: string (nullable = true)



In [0]:
# Render filtered data    
df.where("quantity < 20") \
  .show()

+---------------+--------+---------+-----+-----------+--------------------+
|           name|quantity|clearance|price|         id|            category|
+---------------+--------+---------+-----+-----------+--------------------+
|Yamba Surfboard|      12|    false|  850|68719518391|gear-surf-surfboards|
|Yamba Surfboard|      12|    false|  850|          1|gear-surf-surfboards|
+---------------+--------+---------+-----+-----------+--------------------+



In [0]:
# Render 1 row of flitered data    
df.filter(df.clearance == True) \
  .show(1)

+--------------------+--------+---------+-----+-----------+--------------------+
|                name|quantity|clearance|price|         id|            category|
+--------------------+--------+---------+-----+-----------+--------------------+
|Kiama Classic Sur...|      25|     true|  790|68719518371|gear-surf-surfboards|
+--------------------+--------+---------+-----+-----------+--------------------+
only showing top 1 row



In [0]:
# Render five rows of unfiltered and untruncated data    
df.show(5, False)

+-----------------------+--------+---------+-----+-----------+--------------------+
|name                   |quantity|clearance|price|id         |category            |
+-----------------------+--------+---------+-----+-----------+--------------------+
|Kiama Classic Surfboard|25      |true     |790  |68719518371|gear-surf-surfboards|
|Yamba Surfboard        |12      |false    |850  |68719518391|gear-surf-surfboards|
|Yamba Surfboard        |12      |false    |850  |1          |gear-surf-surfboards|
|Kiama Classic Surfboard|25      |true     |790  |2          |gear-surf-surfboards|
+-----------------------+--------+---------+-----+-----------+--------------------+



In [0]:
# Render results of raw query    
rawQuery = "SELECT * FROM cosmosCatalog.cosmicworks.products WHERE price > 800"
rawDf = spark.sql(rawQuery)
rawDf.show()

+---------------+--------+---------+-----+-----------+--------------------+
|           name|quantity|clearance|price|         id|            category|
+---------------+--------+---------+-----+-----------+--------------------+
|Yamba Surfboard|      12|    false|  850|68719518391|gear-surf-surfboards|
|Yamba Surfboard|      12|    false|  850|          1|gear-surf-surfboards|
+---------------+--------+---------+-----+-----------+--------------------+



In [0]:
# Create employee data
employees = (
  ("6347638858132", "CosmicWorks", "Marketing", "Outside Sales", "Alain Henry",  '[ { "type": "phone", "value": "425-555-0117" }, { "email": "alain@adventure-works.com" } ]'), 
)

In [0]:
# Ingest data

config = {
  "spark.cosmos.accountEndpoint": "https://democosmosnosql.documents.azure.com:443/",
  "spark.cosmos.accountKey": "lRzLNbbsfXgSI4Y7cl6GAqbXITP1GgcNWqRU28MzBVjLJxDBf34rYEmWf9dazOoOHXH56ZZtprHQACDbtMW3Sg==",
#   "spark.cosmos.database": "cosmicworks",
#   "spark.cosmos.container": "employees"
}

spark.createDataFrame(employees) \
  .toDF("id", "organization", "department", "team", "name", "contacts") \
  .write \
  .format("cosmos.oltp") \
  .options(**config) \
  .option("spark.cosmos.database" , "cosmicworks") \
  .option("spark.cosmos.container" , "employees") \
  .mode("APPEND") \
  .save()

In [0]:
# Read and render data
rawJsonDf = spark.read.format("cosmos.oltp") \
  .options(**config) \
  .option("spark.cosmos.database" , "cosmicworks") \
  .option("spark.cosmos.container" , "employees") \
  .load()
rawJsonDf.show()

+------------+-----------+--------------------+----------+-------------+-------------+
|organization|       name|            contacts|department|           id|         team|
+------------+-----------+--------------------+----------+-------------+-------------+
| CosmicWorks|Alain Henry|[ { "type": "phon...| Marketing|  63476388581|Outside Sales|
| CosmicWorks|Alain Henry|[ { "type": "phon...| Marketing|6347638858132|Outside Sales|
+------------+-----------+--------------------+----------+-------------+-------------+

